<a href="https://colab.research.google.com/github/denklewer/ray-custom-agents/blob/master/trpo_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!sudo apt-get install -y build-essential curl unzip psmisc
!pip install cython==0.29.0
!git clone https://github.com/ray-project/ray.git
!ray/ci/travis/install-bazel.sh
!pip install lz4

In [0]:
!mv ray ray-distr
!pip install -e ray-distr/python/. --verbose  # Add --user if you see a permission denied error.

In [0]:
import sys
import os
sys.path.append(os.path.join("", 'ray-distr/python')) # To find local version of the library

In [0]:
import ray.rllib

In [0]:
!python -m pytest -v ray-distr/python/ray/tests/test_mini.py

In [12]:
!git clone https://github.com/denklewer/ray-custom-agents.git


Cloning into 'ray-custom-agents'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 26 (delta 8), reused 23 (delta 5), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [0]:

sys.path.append(os.path.join("", 'ray-custom-agents')) # To find local version of the library


In [23]:
sys.path


['/content/ray/python/ray/pyarrow_files',
 '',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 'ray-custom-agents/ray/python',
 'ray-custom-agents',
 'ray/python',
 '/content/ray/python/ray/modin']

In [16]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import ray
from trpo import TRPOAgent, DEFAULT_CONFIG
from ray.tune.logger import pretty_print


ModuleNotFoundError: ignored

In [7]:
ray.init(ignore_reinit_error=True)

2019-05-17 13:14:57,891	WARNING worker.py:1337 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-17 13:14:57,895	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-17_13-14-57_138/logs.
2019-05-17 13:14:58,020	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:42982 to respond...
2019-05-17 13:14:58,164	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:55473 to respond...
2019-05-17 13:14:58,172	INFO services.py:804 -- Starting Redis shard with 2.58 GB max memory.
2019-05-17 13:14:58,232	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-17_13-14-57_138/logs.
2019-05-17 13:14:58,238	INFO services.py:1427 -- Starting the Plasma object store with 3.87 GB memory using /dev/shm.


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2019-05-17_13-14-57_138/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-17_13-14-57_138/sockets/raylet',
 'redis_address': '172.28.0.2:42982',
 'webui_url': None}

In [11]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 2
config['num_sgd_iter'] = 30
config['num_gpus'] = 3
config['sgd_minibatch_size'] = 128
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

agent = PPOAgent(config, 'CartPole-v0')

2019-05-17 13:18:56,802	WARNING __init__.py:21 -- DeprecationWarning: PPOAgent has been renamed to PPOTrainer. This will raise an error in the future.
2019-05-17 13:18:56,811	WARNING ppo.py:171 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-05-17 13:18:56,815	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-05-17 13:18:58,223	INFO policy_evaluator.py:728 -- Built policy map: {'default_policy': <ray.rllib.agents.ppo.ppo_policy_graph.PPOPolicyGraph object at 0x7f7c393f15c0>}
2019-05-17 13:18:58,225	INFO policy_evaluator.py:729 -- Built preprocessor map: {'default_policy': <ray.

(pid=500) 2019-05-17 13:18:58,271	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=500) 2019-05-17 13:18:58.286550: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
(pid=500) 2019-05-17 13:18:58.287967: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x349d4a0 executing computations on platform Host. Devices:
(pid=500) 2019-05-17 13:18:58.288008: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=500) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=500) Instructions for updating:
(pid=500) Colocations handled automatically by placer.
(pid=500) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/ray/rllib/models/action_dist.p

In [12]:
for i in range(2):
    result = agent.train()
    print(pretty_print(result))

(pid=500) 2019-05-17 13:19:13,514	INFO policy_evaluator.py:437 -- Generating sample batch of size 200
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:308 -- Raw obs from env: { 0: { 'agent0': np.ndarray((4,), dtype=float64, min=-0.031, max=0.005, mean=-0.013)}}
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:309 -- Info return from env: {0: {'agent0': None}}
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:407 -- Preprocessed obs: np.ndarray((4,), dtype=float64, min=-0.031, max=0.005, mean=-0.013)
(pid=500) 2019-05-17 13:19:13,515	INFO sampler.py:411 -- Filtered obs: np.ndarray((4,), dtype=float64, min=-0.031, max=0.005, mean=-0.013)
(pid=500) 2019-05-17 13:19:13,516	INFO sampler.py:525 -- Inputs to compute_actions():
(pid=500) 
(pid=500) { 'default_policy': [ { 'data': { 'agent_id': 'agent0',
(pid=500)                                   'env_id': 0,
(pid=500)                                   'info': None,
(pid=500)                                   'obs': np.ndarray((4,), dtype=f